## Task Decomposition Prompts

In [33]:
import requests
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from openai import OpenAI

In [3]:
requests.get("http://localhost:11434").content

OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

model_name="llama3.2"

In [5]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
#from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_classic.chains import ConversationChain
from langchain_classic.memory import ConversationBufferMemory

In [6]:
# Initialize the Ollama model
llm = ChatOllama(
    model=model_name,   # change to mistral / qwen2.5 if you want
    temperature=0
)

In [9]:
def run_prompt(prompt, **kwargs):
    """Helper function to run a prompt through the language model.

    Args:
        prompt (str): The prompt template string.
        **kwargs: Keyword arguments to fill the prompt template.

    Returns:
        str: The model's response.
    """
    prompt_template = PromptTemplate(template=prompt, input_variables=list(kwargs.keys()))
    chain = prompt_template | llm
    return chain.invoke(kwargs).content

## Breaking Down Complex Tasks

In [13]:
complex_task = """
Analyze the financial health of a company based on the following data:
- Revenue: $10 million
- Net Income: $2 million
- Total Assets: $15 million
- Total Liabilities: $7 million
- Cash Flow from Operations: $3 million
"""

decomposition_prompt = """
Break down the task of analyzing a company's financial health into 3 subtasks. For each subtask, provide a brief description of what it should accomplish.

Task: {task}

Subtasks:
1.
"""

subtasks = run_prompt(decomposition_prompt, task=complex_task)
print(subtasks)

Here are the 3 subtasks to analyze a company's financial health, along with brief descriptions of what each subtask should accomplish:

**Subtask 1: Evaluate Liquidity and Solvency**

This subtask should assess whether the company has sufficient liquidity and solvency to meet its short-term obligations. Key metrics to consider include:

* Cash Flow from Operations: $3 million
* Current Ratio (Current Assets / Current Liabilities): Should be greater than 1, indicating that the company can cover its current liabilities with its liquid assets.
* Debt-to-Equity Ratio (Total Liabilities / Total Shareholders' Equity): Should be reasonable, indicating that the company's debt is manageable.

By evaluating liquidity and solvency, you'll determine whether the company has a solid foundation to meet its financial obligations and can continue to operate without significant disruptions.

**Subtask 2: Assess Profitability**

This subtask should evaluate the company's ability to generate profits and m

## Chaining Subtasks in Prompts

In [16]:
def analyze_profitability(revenue, net_income):
    """Analyze the company's profitability.

    Args:
        revenue (float): Company's revenue.
        net_income (float): Company's net income.

    Returns:
        str: Analysis of the company's profitability.
    """
    prompt = """
    Analyze the company's profitability based on the following data:
    - Revenue: ${revenue} million
    - Net Income: ${net_income} million

    Calculate the profit margin and provide a brief analysis of the company's profitability.
    """
    return run_prompt(prompt, revenue=revenue, net_income=net_income)

def analyze_liquidity(total_assets, total_liabilities):
    """Analyze the company's liquidity.

    Args:
        total_assets (float): Company's total assets.
        total_liabilities (float): Company's total liabilities.

    Returns:
        str: Analysis of the company's liquidity.
    """
    prompt = """
    Analyze the company's liquidity based on the following data:
    - Total Assets: ${total_assets} million
    - Total Liabilities: ${total_liabilities} million

    Calculate the current ratio and provide a brief analysis of the company's liquidity.
    """
    return run_prompt(prompt, total_assets=total_assets, total_liabilities=total_liabilities)

def analyze_cash_flow(cash_flow):
    """Analyze the company's cash flow.

    Args:
        cash_flow (float): Company's cash flow from operations.

    Returns:
        str: Analysis of the company's cash flow.
    """
    prompt = """
    Analyze the company's cash flow based on the following data:
    - Cash Flow from Operations: ${cash_flow} million

    Provide a brief analysis of the company's cash flow health.
    """
    return run_prompt(prompt, cash_flow=cash_flow)

# Run the chained subtasks
profitability_analysis = analyze_profitability(10, 2)
liquidity_analysis = analyze_liquidity(15, 7)
cash_flow_analysis = analyze_cash_flow(3)

print("Profitability Analysis:\n", profitability_analysis)
print("\nLiquidity Analysis:\n", liquidity_analysis)
print("\nCash Flow Analysis:\n", cash_flow_analysis)

Profitability Analysis:
 To calculate the profit margin, we need to use the formula:

Profit Margin = (Net Income / Revenue) x 100

Plugging in the given values:

Profit Margin = ($2 million / $10 million) x 100
= 0.20 x 100
= 20%

Based on this calculation, the company's profit margin is 20%.

A profit margin of 20% indicates that for every dollar of revenue generated, the company retains 20 cents as net income. This suggests that the company has a relatively healthy profitability profile.

However, it's essential to consider other factors when evaluating a company's overall financial health, such as:

1. Operating expenses: Are they manageable and aligned with industry standards?
2. Debt levels: Is the company's debt-to-equity ratio reasonable?
3. Growth prospects: Is the company growing revenue and expanding its market share?

While a 20% profit margin is a good starting point, it's crucial to examine these additional factors to gain a more comprehensive understanding of the company

In [18]:
def integrate_results(profitability, liquidity, cash_flow):
    """Integrate the results from subtasks to provide an overall analysis.

    Args:
        profitability (str): Profitability analysis.
        liquidity (str): Liquidity analysis.
        cash_flow (str): Cash flow analysis.

    Returns:
        str: Overall analysis of the company's financial health.
    """
    prompt = """
    Based on the following analyses, provide an overall assessment of the company's financial health:

    Profitability Analysis:
    {profitability}

    Liquidity Analysis:
    {liquidity}

    Cash Flow Analysis:
    {cash_flow}

    Summarize the key points and give an overall evaluation of the company's financial position.
    """
    return run_prompt(prompt, profitability=profitability, liquidity=liquidity, cash_flow=cash_flow)

overall_analysis = integrate_results(profitability_analysis, liquidity_analysis, cash_flow_analysis)
print("Overall Financial Health Analysis:\n", overall_analysis)

Overall Financial Health Analysis:
 Based on the provided analyses, here are the key points:

1. Profitability: The company has a relatively healthy profit margin of 20%, indicating that it retains a significant portion of its revenue as net income.
2. Liquidity: The debt-to-equity ratio is moderate (46.7%), and assuming significant total current assets, the current ratio is strong (114.3%). However, without more information, this analysis should be considered rough estimates rather than exact assessments.
3. Cash Flow: The company has generated $3 million in cash flow from operations, which suggests sufficient liquidity to meet short-term obligations.

Overall Evaluation:

The company appears to have a relatively healthy financial profile, with a strong profit margin and moderate debt levels. However, there are some areas of concern that require further investigation:

* The lack of information on operating expenses makes it difficult to assess the company's ability to manage costs an

# Prompt Chaining and Subsequences

## Basic Prompt Chaining

In [21]:
# Define prompt templates
story_prompt = PromptTemplate(
    input_variables=["genre"],
    template="Write a short {genre} story in 3-4 sentences."
)

summary_prompt = PromptTemplate(
    input_variables=["story"],
    template="Summarize the following story in one sentence:\n{story}"
)

# Chain the prompts
def story_chain(genre):
    """Generate a story and its summary based on a given genre.

    Args:
        genre (str): The genre of the story to generate.

    Returns:
        tuple: A tuple containing the generated story and its summary.
    """
    story = (story_prompt | llm).invoke({"genre": genre}).content
    summary = (summary_prompt | llm).invoke({"story": story}).content
    return story, summary

# Test the chain
genre = "science fiction"
story, summary = story_chain(genre)
print(f"Story: {story}\n\nSummary: {summary}")

Story: As the last star in the universe died, Captain Jaxon gazed out at the endless expanse of darkness, his ship's engines humming softly as it hurtled through the void. The crew had long since succumbed to the crushing loneliness, but Jaxon remained, driven by a stubborn determination to find a new home for humanity. Suddenly, a faint signal flickered on the horizon - a message from an unknown source, beckoning them towards a distant planet that might hold the key to their survival. With newfound hope, Jaxon steered his ship towards the light, and the universe's fate hung in the balance.

Summary: As the last star in the universe died, Captain Jaxon received a mysterious signal from an unknown source, leading him to a distant planet that might hold the key to humanity's survival.


## Sequential Prompting

In [25]:
# Define prompt templates for each step
theme_prompt = PromptTemplate(
    input_variables=["text"],
    template="Identify the main theme of the following text:\n{text}"
)

tone_prompt = PromptTemplate(
    input_variables=["text"],
    template="Describe the overall tone of the following text:\n{text}"
)

takeaway_prompt = PromptTemplate(
    input_variables=["text", "theme", "tone"],
    template="Given the following text with the main theme '{theme}' and tone '{tone}', what are the key takeaways?\n{text}"
)

def analyze_text(text):
    """Perform a multi-step analysis of a given text.

    Args:
        text (str): The text to analyze.

    Returns:
        dict: A dictionary containing the theme, tone, and key takeaways of the text.
    """
    theme = (theme_prompt | llm).invoke({"text": text}).content
    tone = (tone_prompt | llm).invoke({"text": text}).content
    takeaways = (takeaway_prompt | llm).invoke({"text": text, "theme": theme, "tone": tone}).content
    return {"theme": theme, "tone": tone, "takeaways": takeaways}

# Test the sequential prompting
sample_text = "The rapid advancement of artificial intelligence has sparked both excitement and concern among experts. While AI promises to revolutionize industries and improve our daily lives, it also raises ethical questions about privacy, job displacement, and the potential for misuse. As we stand on the brink of this technological revolution, it's crucial that we approach AI development with caution and foresight, ensuring that its benefits are maximized while its risks are minimized."

analysis = analyze_text(sample_text)
for key, value in analysis.items():
    print(f"{key.capitalize()}: {value}\n")

Theme: The main theme of the text is the need to balance the benefits and risks associated with the rapid advancement of artificial intelligence (AI).

Tone: The overall tone of the text is cautious and reflective. The language used conveys a sense of awareness about the potential consequences of artificial intelligence and acknowledges both the excitement and concerns surrounding its advancement. The use of words like "caution" and "foresight" suggests a measured and thoughtful approach, implying that the author is urging readers to consider the implications of AI development carefully.

There is also a slightly neutral tone, as the text does not express a strong emotional stance or advocate for a particular course of action. Instead, it presents a balanced view, highlighting both the potential benefits and risks of AI, and encouraging readers to approach the topic with careful consideration. Overall, the tone is informative, analytical, and advisory, suggesting that the author is see

## Dynamic Prompt Generation

In [28]:
# Define prompt templates
answer_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question concisely:\n{question}"
)

follow_up_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Based on the question '{question}' and the answer '{answer}', generate a relevant follow-up question."
)

def dynamic_qa(initial_question, num_follow_ups=3):
    """Conduct a dynamic Q&A session with follow-up questions.

    Args:
        initial_question (str): The initial question to start the Q&A session.
        num_follow_ups (int): The number of follow-up questions to generate.

    Returns:
        list: A list of dictionaries containing questions and answers.
    """
    qa_chain = []
    current_question = initial_question

    for _ in range(num_follow_ups + 1):  # +1 for the initial question
        answer = (answer_prompt | llm).invoke({"question": current_question}).content
        qa_chain.append({"question": current_question, "answer": answer})
        
        if _ < num_follow_ups:  # Generate follow-up for all but the last iteration
            current_question = (follow_up_prompt | llm).invoke({"question": current_question, "answer": answer}).content

    return qa_chain

# Test the dynamic Q&A system
initial_question = "What are the potential applications of quantum computing?"
qa_session = dynamic_qa(initial_question)

for i, qa in enumerate(qa_session):
    print(f"Q{i+1}: {qa['question']}")
    print(f"A{i+1}: {qa['answer']}\n")

Q1: What are the potential applications of quantum computing?
A1: The potential applications of quantum computing include:

1. Cryptography and cybersecurity
2. Optimization problems (e.g., logistics, finance)
3. Materials science and chemistry
4. Artificial intelligence and machine learning
5. Simulations (e.g., weather forecasting, medical research)
6. Data analysis and pattern recognition
7. Machine learning and predictive modeling

These applications leverage the unique properties of quantum computing, such as superposition and entanglement, to solve complex problems more efficiently than classical computers.

Q2: "Can you elaborate on how quantum computing's unique properties of superposition and entanglement are being leveraged in each of these applications, and what specific challenges or limitations are being addressed by the use of quantum computing?"
A2: Quantum computing's unique properties of superposition and entanglement are being leveraged in various applications to solv

## Error Handling and Validation

In [37]:
import re

In [39]:
# Define prompt templates
generate_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Generate a 4-digit number related to the topic: {topic}. Respond with ONLY the number, no additional text."
)

validate_prompt = PromptTemplate(
    input_variables=["number", "topic"],
    template="Is the number {number} truly related to the topic '{topic}'? Answer with 'Yes' or 'No' and explain why."
)

def extract_number(text):
    """Extract a 4-digit number from the given text.

    Args:
        text (str): The text to extract the number from.

    Returns:
        str or None: The extracted 4-digit number, or None if no valid number is found.
    """
    match = re.search(r'\b\d{4}\b', text)
    return match.group() if match else None

def robust_number_generation(topic, max_attempts=3):
    """Generate a topic-related number with validation and error handling.

    Args:
        topic (str): The topic to generate a number for.
        max_attempts (int): Maximum number of generation attempts.

    Returns:
        str: A validated 4-digit number or an error message.
    """
    for attempt in range(max_attempts):
        try:
            response = (generate_prompt | llm).invoke({"topic": topic}).content
            number = extract_number(response)
            
            if not number:
                raise ValueError(f"Failed to extract a 4-digit number from the response: {response}")
            
            # Validate the relevance
            validation = (validate_prompt | llm).invoke({"number": number, "topic": topic}).content
            if validation.lower().startswith("yes"):
                return number
            else:
                print(f"Attempt {attempt + 1}: Number {number} was not validated. Reason: {validation}")
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
    
    return "Failed to generate a valid number after multiple attempts."

# Test the robust number generation
topic = "World War II"
result = robust_number_generation(topic)
print(f"Final result for topic '{topic}': {result}")

Final result for topic 'World War II': 1945


# Instruction Engineering 

In [42]:
def get_completion(prompt):
    """Helper function to get model completion."""
    return llm.invoke(prompt).content

In [44]:
vague_instruction = "Tell me about climate change concisely."
clear_instruction = "Provide a concise summary of the primary causes and effects of climate change, focusing on scientific consensus from the past five years concisely."

print("Vague Instruction Output:")
print(get_completion(vague_instruction))

print("\nClear Instruction Output:")
print(get_completion(clear_instruction))

Vague Instruction Output:
Here's a concise overview of climate change:

**What is climate change?**

Climate change refers to the long-term warming of the planet due to an increase in average global temperatures, primarily caused by human activities that release greenhouse gases (GHGs) into the atmosphere.

**Causes:**

1. Burning fossil fuels (coal, oil, gas)
2. Deforestation and land-use changes
3. Agriculture and livestock production
4. Industrial processes

**Effects:**

1. Rising sea levels and coastal flooding
2. More frequent and severe heatwaves, droughts, and storms
3. Changes in precipitation patterns and water scarcity
4. Loss of biodiversity and ecosystem disruption

**Consequences:**

1. Sea-level rise (up to 1 meter by 2100)
2. Increased risk of extreme weather events
3. Negative impacts on human health, economy, and food security
4. Irreversible damage to ecosystems and natural habitats

**What can we do?**

1. Reduce GHG emissions through renewable energy, energy effici

## Effective Instruction Structures

In [47]:
bullet_structure = """
Explain the process of photosynthesis concisely:
- Define photosynthesis
- List the main components involved
- Describe the steps in order
- Mention its importance for life on Earth
"""

narrative_structure = """
Imagine you're a botanist explaining photosynthesis to a curious student. 
Start with a simple definition, then walk through the process step-by-step, 
highlighting the key components involved. Conclude by emphasizing why 
photosynthesis is crucial for life on Earth. Write it concisely.
"""

print("Bullet Structure Output:")
print(get_completion(bullet_structure))

print("\nNarrative Structure Output:")
print(get_completion(narrative_structure))

Bullet Structure Output:
**Definition:** Photosynthesis is a biological process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of organic compounds, such as glucose.

**Main Components:**

1. Chlorophyll (green pigment) in plant cells
2. Water (H2O)
3. Carbon dioxide (CO2)
4. Light energy from the sun

**Steps:**

1. **Light absorption**: Chlorophyll absorbs light energy from the sun, which excites electrons.
2. **Water absorption**: Plants absorb water through their roots.
3. **Carbon fixation**: CO2 is fixed into a three-carbon molecule called 3-phosphoglycerate (3-PGA) via the enzyme RuBisCO.
4. **Calvin cycle**: The energy from light is used to convert 3-PGA into glucose (C6H12O6).
5. **Oxygen release**: Oxygen (O2) is released as a byproduct.

**Importance:**

Photosynthesis is crucial for life on Earth, as it:

1. Produces oxygen, which supports the respiratory needs of most living organisms.
2. Provides energy and org

## Balancing Specificity and Generality

In [50]:
specific_instruction = """
Describe the plot of the 1985 film 'Back to the Future', focusing on:
1. The main character's name and his friendship with Dr. Brown
2. The time machine and how it works
3. The specific year the main character travels to and why it's significant
4. The main conflict involving his parents' past
5. How the protagonist resolves the issues and returns to his time
Limit your response to 150 words. 
"""

general_instruction = """
Describe the plot of a popular time travel movie from the 1980s. Include:
1. The main characters and their relationships
2. The method of time travel
3. The time period visited and its significance
4. The main conflict or challenge faced
5. How the story is resolved
Keep your response around 150 words.
"""

print("Specific Instruction Output:")
print(get_completion(specific_instruction))

print("\nGeneral Instruction Output:")
print(get_completion(general_instruction))

Specific Instruction Output:
In 'Back to the Future', Marty McFly, a high school student, befriends eccentric scientist Doc Brown. They build a time-traveling DeLorean, which uses 1.21 gigawatts of electricity to break the time barrier.

Marty travels back to November 5, 1955, where he meets his parents when they were his age. However, his presence in the past causes his parents to fall in love and get married, altering their future. Marty must ensure his parents meet and fall in love naturally.

To resolve the issue, Marty returns to 1985 and convinces his parents to attend a dance, where they meet each other. He also ensures that his mother's guitar-playing skills are showcased at the dance, which helps her win over his father. With the timeline restored, Marty returns to 1955, and his parents' love story unfolds as it should.

General Instruction Output:
The plot of "Back to the Future" (1985) revolves around Marty McFly, a high school student played by Michael J. Fox, who travels b

## Iterative Refinement

In [ ]:
initial_instruction = "Explain how to make a peanut butter and jelly sandwich."

print("Initial Instruction Output:")
initial_output = get_completion(initial_instruction)
print(initial_output)

refined_instruction = """
Explain how to make a peanut butter and jelly sandwich, with the following improvements:
1. Specify the type of bread, peanut butter, and jelly to use
2. Include a step about washing hands before starting
3. Mention how to deal with potential allergies
4. Add a tip for storing the sandwich if not eaten immediately
Present the instructions in a numbered list format.
"""

print("\nRefined Instruction Output:")
refined_output = get_completion(refined_instruction)
print(refined_output)

Initial Instruction Output:
